In [1]:
import mysql.connector
import pandas as pd 
import json
import numpy as np
import MySQLdb
import MySQLdb.cursors

In [2]:
def get_soal(paket_id):
    sql = "SELECT * FROM soal where paket_id='%s'" % (paket_id)
    soal = pd.read_sql(sql, con=db_thread)
    soal = soal.set_index('id') 
    return soal

In [3]:
def get_ujian_siswa(paket_id):
    sql = "select * from ujian_siswa where paket_id='%s' and (status=2)" % (paket_id)
    cur_thread.execute(sql)
    ujian_siswa = cur_thread.fetchall()
    return ujian_siswa

In [4]:
def init_load_ujian(random_soal, random_jawaban, jawaban_siswa):
    soals = json.loads(random_soal)
    jawaban_posisi = json.loads(random_jawaban)
    jawabans = json.loads(jawaban_siswa)
    return soals, jawaban_posisi, jawabans

In [5]:
def init_posisi_jawaban():
    intochar={'1':"a",'2':"b",'3':"c",'4':"d",'5':"e"}
    arr_random= {
        "a" : intochar[jawaban_posisi[key][0]],
        "b" : intochar[jawaban_posisi[key][1]],
        "c" : intochar[jawaban_posisi[key][2]],
        "d" : intochar[jawaban_posisi[key][3]],
        "e" : intochar[jawaban_posisi[key][4]],
    }
    return arr_random

In [6]:
def sebaran_jawaban(arr_random, soal):
    if(arr_random[jawabans[key].lower()] == 'a'):
        soal.loc[soal_id, 'jawaban_a'] += 1 
    elif(arr_random[jawabans[key].lower()] == 'b'):
        soal.loc[soal_id, 'jawaban_b'] += 1
    elif(arr_random[jawabans[key].lower()] == 'c'): 
        soal.loc[soal_id, 'jawaban_c'] += 1
    elif(arr_random[jawabans[key].lower()] == 'd'):
        soal.loc[soal_id, 'jawaban_d'] += 1
    elif(arr_random[jawabans[key].lower()] == 'e'):
        soal.loc[soal_id, 'jawaban_e'] += 1

In [7]:
def update_hasil(soal, paket_id):
    for index, row in soal.iterrows():
        sql = "update soal set jawaban_a=%d, jawaban_b=%d, jawaban_c=%d, jawaban_d=%d, jawaban_e=%d, siswa_id_benar='%s', jumlah_benar_siswa=%d, jumlah_siswa=%d where id='%s'" %(row['jawaban_a'], row['jawaban_b'], row['jawaban_c'], row['jawaban_d'], row['jawaban_e'], row['siswa_id_benar'], row['jumlah_benar_siswa'], row['jumlah_siswa'], index)
        cur_thread.execute(sql)
    sql = "update ujian_siswa set status=3 where paket_id='%s' and (status=2)" % (paket['id'])
    cur_thread.execute(sql)
    sql = "update paket set status=1 where id='%s'" % (paket['id'])
    cur_thread.execute(sql)
    db_thread.commit()

In [8]:
db_thread = MySQLdb.connect("127.0.0.1", "root", "andhika", "usek_monev", cursorclass=MySQLdb.cursors.DictCursor)
cur_thread = db_thread.cursor()
print(cur_thread)

In [ ]:
cur_thread.execute("SELECT * from paket where status = 0")
pakets = cur_thread.fetchall()
for paket in pakets:
    soal = get_soal(paket['id'])
    ujian_siswas = get_ujian_siswa(paket['id'])
    print(len(ujian_siswas))
    for ujian_siswa in ujian_siswas:
        soals, jawaban_posisi, jawabans = init_load_ujian(ujian_siswa["random_soal"], ujian_siswa["random_jawaban"], ujian_siswa["jawaban_siswa"])
        for key, soal_id in enumerate(soals):
            siswa_id_benar = json.loads(soal.loc[soal_id, 'siswa_id_benar'])
            if (soal.loc[soal_id, 'tipe_soal'] == 'pilihan_ganda'): 
                arr_random = init_posisi_jawaban()
                if(jawabans[key] ==" "):
                    jawaban_benar=0
                else:
                    if (arr_random[jawabans[key].lower()] == soal.loc[soal_id, 'kunci_jawaban'].lower()): 
                        jawaban_benar=1
                        siswa_id_benar.append(ujian_siswa['id'])
                        s = json.dumps(siswa_id_benar)
                        soal.loc[soal_id, 'siswa_id_benar'] = s
                    else :
                        jawaban_benar=0                
                    sebaran_jawaban(arr_random, soal)
                soal.loc[soal_id, 'jumlah_benar_siswa'] += jawaban_benar                
                soal.loc[soal_id, 'jumlah_siswa'] += 1
    update_hasil(soal, paket['id'])
    print("~DONE~ paket:", paket['nama'])

24687
~DONE~ paket: K13_Bahasa_Inggris_1
24695
~DONE~ paket: K13_Bahasa_Inggris_2
15175
~DONE~ paket: K13_Bahasa_Inggris_3
15365
~DONE~ paket: K13_Bahasa_Inggris_4
0
~DONE~ paket: K13_Bahasa_Inggris_5
0
~DONE~ paket: K13_Bahasa_dan_Sastra_Inggris_(Peminatan)_1
12426
~DONE~ paket: K13_Bahasa_dan_Sastra_Inggris_(Peminatan)_2
1055
~DONE~ paket: K13_Bahasa_dan_Sastra_Inggris_(Peminatan)_3
8485
~DONE~ paket: K13_Bahasa_dan_Sastra_Inggris_(Peminatan)_4
0
~DONE~ paket: K13_Bahasa_dan_Sastra_Inggris_(Peminatan)_5
24127
~DONE~ paket: K13_Sejarah_Indonesia_1
24082
~DONE~ paket: K13_Sejarah_Indonesia_2
14832
~DONE~ paket: K13_Sejarah_Indonesia_3
14868
~DONE~ paket: K13_Sejarah_Indonesia_4
905
~DONE~ paket: K13_Sejarah_Indonesia_5
8804
~DONE~ paket: K13_Sejarah_Peminatan_IPS_1
8902
~DONE~ paket: K13_Sejarah_Peminatan_IPS_2
8513
~DONE~ paket: K13_Sejarah_Peminatan_IPS_3
8511
~DONE~ paket: K13_Sejarah_Peminatan_IPS_4
0
~DONE~ paket: K13_Sejarah_Peminatan_IPS_5
11577
~DONE~ paket: K13_Seni_Rupa_4
0
~